In [1]:
from symmer import PauliwordOp, QuantumState
import qiskit
import numpy as np
from basis_states import *
from hamiltonian import *

In [2]:
N = 6

In [3]:
H = H(N)

In [4]:
mesons, baryons = get_basis(N)

In [5]:
print("To map to the meson sector, we decrease from ", 2*N, "qubits to ", int(np.ceil(np.log2(len(mesons)))))
print("To map to the baryon sector, we decrease from ", 2*N, "qubits to ", int(np.ceil(np.log2(len(baryons)))))

To map to the meson sector, we decrease from  12 qubits to  9
To map to the baryon sector, we decrease from  12 qubits to  8


In [6]:
def one_hot_to_binary_encoding(states):
    new_nqubits = int(np.ceil(np.log2(len(states))))
    return {format(i, '0' + str(new_nqubits) + 'b'):states[i] for i in range(len(states))}

In [7]:
def H_sector(hamiltonian, states):
    size = len(states)
    H_sector_matrix = np.zeros((size, size), dtype = complex)
    
    for i in range(len(states)):
        for j in range(len(states)):
            H_sector_matrix[i][j] = states[i].dagger * hamiltonian * states[j]
    return PauliwordOp.from_matrix(H_sector_matrix)

In [8]:
oh_binary_dict = one_hot_to_binary_encoding(mesons)

In [9]:
H_meson = H_sector(H, mesons)

/Users/carterm.gustin/anaconda3/envs/phd/lib/python3.9/site-packages/symmer/operators/base.py:400: ComplexWarning: Casting complex values to real discards the imaginary part
  temp_mat[:matrix.shape[0],


Building operator via projectors diag elements:   0%|          | 0/399 [00:00<?, ?it/s]

Building operator via projectors off-diag elements:   0%|          | 0/3120 [00:00<?, ?it/s]

In [10]:
#meson_sector
vals, vecs = np.linalg.eigh(H_meson.to_sparse_matrix.toarray())

index = 0
ground_state = QuantumState.from_array(vecs[:, index].reshape([-1, 1]))
ground_energy = vals[index]

In [11]:
psi = ground_state.sort('decreasing').cleanup(1e-3)
psi_OH = QuantumState([0]*H.n_qubits)*0

for state in psi.to_dictionary:
    psi_OH += oh_binary_dict[state] * psi.to_dictionary[state]


In [12]:

psi_OH.sort('decreasing')

 0.629+0.000j |100100110011> +
-0.629+0.000j |011000110011> +
-0.136+0.000j |100101100011> +
-0.136+0.000j |011010010011> +
-0.128+0.000j |100100110110> +
-0.128+0.000j |011000111001> +
 0.125+0.000j |011000110110> +
 0.125+0.000j |100100111001> +
 0.125+0.000j |011001100011> +
 0.125+0.000j |100110010011> +
-0.123+0.000j |100100011011> +
-0.123+0.000j |011000100111> +
 0.121+0.000j |011000011011> +
 0.121+0.000j |100100100111> +
 0.031+0.000j |100101100110> +
-0.031+0.000j |011010011001> +
-0.030+0.000j |100101001011> +
-0.030+0.000j |011010000111> +
 0.030+0.000j |100111000011> +
-0.030+0.000j |011011000011> +
-0.029+0.000j |011000111100> +
 0.029+0.000j |100100111100> +
-0.028+0.000j |011001100110> +
 0.028+0.000j |100110011001> +
 0.027+0.000j |011001001011> +
 0.027+0.000j |100110000111> +
-0.026+0.000j |011000001111> +
 0.026+0.000j |100100001111> +
-0.026+0.000j |100101101001> +
 0.026+0.000j |011010010110> +
 0.026+0.000j |100100011110> +
 0.026+0.000j |011000101101> +
-0.025+0